In [1]:
import gurobipy as gp
import sys
from gurobipy import GRB
import numpy as np
import pandas as pd

In [27]:
cri = 5
dm = 3
alter = 6
# data = []
# for i in range(dm):
#     data.append([])
#     for j in range(cri):
#         matrix = np.random.randint(1,7,size=(alter,alter)) #生成随机数据
#         matrix = np.triu(matrix, k=0) #下三角为0
#         np.fill_diagonal(matrix, 0) #对角线为0
#         data[i].append(matrix)
# data
# r_data = pd.read_excel('E:\正在构思或撰写的论文\会议论文\data.xlsx')
r_data =pd.read_excel('E:\正在构思或撰写的论文\会议论文\data.xlsx',sheet_name='final')
data = {}
for i in range(5):
    data[i] = r_data.iloc[0+(i)*7:6+(i)*7]

In [31]:
#EJOR 
inp = data[3].values
model = gp.Model()
#变量

at = model.addVars(6,lb=0,vtype=gp.GRB.CONTINUOUS,name="Attractiveness threshold")
g = model.addVars(alter+1,lb=0,vtype=gp.GRB.CONTINUOUS,name="Alternatives performance")
deta = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="beta")
ibci = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="ibci")
alpha = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="alpha")
eta = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="eta")

c = 0
#目标函数
#     MACBETH_criteira(data[0][0],alter)
obj = model.setObjective(eta.sum("*")+ibci.sum("*")+alpha.sum("*"),GRB.MINIMIZE)

#限制条件
model.addConstr(at[0]==0)
# model.addConstr(at[1]==1)
model.addConstr(g[1]==0)
model.addConstr(g[alter]==100)
for i in range(1,6):
    model.addConstr(at[i]-at[i-1]>=1)

for i in range(1,alter+1):
    for j in range(i+1,alter+1):
        model.addConstr(g[alter-i+1]-g[alter-j+1]>= at[inp[i-1,j-1]-1]-c+1)
#         model.addConstr(g[alter-i+1]-g[alter-j+1]>= theta)
        if inp[i-1,j-1]<= 5:
            model.addConstr(g[alter-i+1]-g[alter-j+1]<= at[inp[i-1,j-1]]+c-1)
            model.addConstr(g[alter-i+1]-g[alter-j+1]== (at[inp[i-1,j-1]-1]+at[inp[i-1,j-1]])/2+ibci[alter-i,alter-j]-eta[alter-i,alter-j])
        else:
            model.addConstr(g[alter-i+1]-g[alter-j+1]== at[inp[i-1,j-1]-1]-alpha[alter-i,alter-j]+deta[alter-i,alter-j])
            

#求解
model.update()  
model.Params.LogToConsole=True # 显示求解过程
model.Params.NonConvex = 1  # 默认求解器，改为 2 时可以解决非凸二次优化问题
model.optimize()
model.getVars()
       
       
       
       
       

Set parameter NonConvex to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 52 rows, 157 columns and 189 nonzeros
Model fingerprint: 0xcee8c5ab
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 8 rows and 120 columns
Presolve time: 0.00s
Presolved: 44 rows, 37 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.226125e+01   0.000000e+00      0s
      25    4.7666667e+01   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.766666667e+01


[<gurobi.Var Attractiveness threshold[0] (value 0.0)>,
 <gurobi.Var Attractiveness threshold[1] (value 19.0)>,
 <gurobi.Var Attractiveness threshold[2] (value 34.333333333333336)>,
 <gurobi.Var Attractiveness threshold[3] (value 59.0)>,
 <gurobi.Var Attractiveness threshold[4] (value 74.33333333333331)>,
 <gurobi.Var Attractiveness threshold[5] (value 99.0)>,
 <gurobi.Var Alternatives performance[0] (value 0.0)>,
 <gurobi.Var Alternatives performance[1] (value 0.0)>,
 <gurobi.Var Alternatives performance[2] (value 13.333333333333343)>,
 <gurobi.Var Alternatives performance[3] (value 40.0)>,
 <gurobi.Var Alternatives performance[4] (value 66.66666666666666)>,
 <gurobi.Var Alternatives performance[5] (value 86.66666666666666)>,
 <gurobi.Var Alternatives performance[6] (value 100.0)>,
 <gurobi.Var beta[0,0] (value 0.0)>,
 <gurobi.Var beta[0,1] (value 0.0)>,
 <gurobi.Var beta[0,2] (value 0.0)>,
 <gurobi.Var beta[0,3] (value 0.0)>,
 <gurobi.Var beta[0,4] (value 0.0)>,
 <gurobi.Var beta[0,5]

In [ ]:
def MACBETH_criteira(data,alter):
    model = gp.Model()
    #变量
    theta = model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name="theta")
    c = model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name="c")
    at = model.addVars(6,lb=0,vtype=gp.GRB.CONTINUOUS,name="Attractiveness threshold")
    g = model.addVars(alter+1,lb=0,vtype=gp.GRB.CONTINUOUS,name="Alternatives performance")

    #目标函数

    obj = model.setObjective(c,GRB.MINIMIZE)
    
    #限制条件
    model.addConstr(at[0]==0)
    model.addConstr(at[1]==1)
    model.addConstr(g[1]==0)
#     model.addConstr(g[alter]==100)
    for i in range(1,6):
        model.addConstr(at[i]-at[i-1]>=1)
        
    for i in range(1,alter+1):
        for j in range(i+1,alter+1):
            model.addConstr(g[alter-i+1]-g[alter-j+1]>= at[data[i-1,j-1]-1]-c+theta)
            model.addConstr(g[alter-i+1]-g[alter-j+1]>= theta)
            if data[i-1,j-1]<= 5:
                model.addConstr(g[alter-i+1]-g[alter-j+1]<= at[data[i-1,j-1]]+c)
#             else:
#                 model.addConstr(g[alter-i+1]-g[alter-j+1]== at[data[i-1,j-1]]+c)
                
    #求解
    model.update()  
    model.Params.LogToConsole=False # 显示求解过程
    # model.Params.MIPGap=0.0001 # 百分比界差
    # model.Params.TimeLimit=100 # 限制求解时间为 100s
    # model.Params.Presolve = -1 # 预处理程度, 0关闭,1保守,2激进
    # model.Params.MIPFocus = 0 # 求解侧重点. 1快速找到可行解, 2证明最有, 3侧重边界提升, 0均衡搜索
    # model.Params.SolutionLimit = inf # 求解数量, 默认求所有解, 比较出最优的结果, 只需要可行解时可以设置该参数为1
    model.Params.NonConvex = 1  # 默认求解器，改为 2 时可以解决非凸二次优化问题
    model.optimize()
    return at,g,c,theta,model.getVars, model.ObjVal
# inp = np.array([[0,3,4,4,5,6],[0,0,1,2,3,4],[0,0,0,1,3,4],[0,0,0,0,2,3],[0,0,0,0,0,1],[0,0,0,0,0,0]])
MACBETH_criteira(inp,alter)

In [ ]:
# 1994 ITOR
model = gp.Model()
#变量
theta = model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name="theta")
at = model.addVars(6,lb=0,vtype=gp.GRB.CONTINUOUS,name="Attractiveness threshold")
g = model.addVars(alter+1,lb=0,vtype=gp.GRB.CONTINUOUS,name="Alternatives performance")
deta = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="beta")
ibci = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="ibci")
alpha = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="alpha")
eta = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="eta")
gamma = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="gamma")
beta = model.addVars(6,6,lb=0,vtype=gp.GRB.CONTINUOUS,name="gamma")

c = 0
#目标函数
#     MACBETH_criteira(data[0][0],alter)
obj = model.setObjective(eta.sum("*")+ibci.sum("*")+alpha.sum("*"),GRB.MINIMIZE)

#限制条件
model.addConstr(at[0]==0)#C1
model.addConstr(at[1]==1) #C1
model.addConstr(g[1]==0)#C4
# model.addConstr(g[alter]==100)
for i in range(2,6):
    model.addConstr(at[i]-at[i-1]>=1) #C2

for i in range(1,alter+1):
    for j in range(i+1,alter+1):
        model.addConstr(g[alter-i+1]-g[alter-j+1]>= theta) #C3
        
        model.addConstr(g[alter-i+1]-g[alter-j+1]>= at[inp[i-1,j-1]-1]-c+theta) #C5, C6
        
#         if inp[i-1,j-1]>=1:
#             model.addConstr(g[alter-i+1]-g[alter-j+1] == at[inp[i-1,j-1]-1]-alpha[alter-i,alter-j]+deta[alter-i,alter-j]+theta) #C8
        if inp[i-1,j-1]<= 5:
            model.addConstr(g[alter-i+1]-g[alter-j+1]<= at[inp[i-1,j-1]]+c-theta) #C5
            model.addConstr(g[alter-i+1]-g[alter-j+1]== (at[inp[i-1,j-1]-1]+at[inp[i-1,j-1]]+theta)/2+ibci[alter-i,alter-j]-eta[alter-i,alter-j]) #C9
            model.addConstr(g[alter-i+1]-g[alter-j+1]== at[inp[i-1,j-1]]+beta[alter-i,alter-j]-gamma[alter-i,alter-j])#C7
        else:
            model.addConstr(g[alter-i+1]-g[alter-j+1]== at[inp[i-1,j-1]-1]-alpha[alter-i,alter-j]+deta[alter-i,alter-j]+theta)
#求解
model.update()  
model.Params.LogToConsole=True # 显示求解过程
model.Params.NonConvex = 1  # 默认求解器，改为 2 时可以解决非凸二次优化问题
model.optimize()
model.getVars()
        

       
       
       
       
       